# Lecture 6: Template attacks


#### Learning goals
- Learn the mathematical backgrounds of _Template Attacks_
- Learn how to perform a Template attack
- Learn how to perform a Template attack using lascar

#### References
- https://wiki.newae.com/Template_Attacks

## 1. Template attack at a glance

Template attacks are a powerful type of side-channel attack. These attacks are a subset of profiling attacks, where an attacker creates a "profile" of a sensitive device and applies this profile to quickly find a victim's secret key.

There are four steps to a template attack:

1. Using a copy of the protected device, record a large number of power traces using many different inputs (plaintexts and keys). Ensure that enough traces are recorded to give us information about each subkey value.
2. Create a template of the device's operation. This template notes a few "points of interest" in the power traces and a multivariate distribution of the power traces at each point.
3. On the victim device, record a small number of power traces. Use multiple plaintexts. (We have no control over the secret key, which is fixed.)
4. Apply the template to the attack traces. For each subkey, track which value is most likely to be the correct subkey. Continue until the key has been recovered.

![Prinzip eines Templateangriffs](./2022-10-06%20Hackerkiste%20Prinzip%20eines%20Templateangriffs.png)

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import numpy as np
import plotly.graph_objects as pgo
import scipy
import lascar

from securec.capture import capture

## 2. Detecting data between noise

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 1</div>

Generate traces using the following code:

```python
data = capture(
    ...
    code="volatile uint8_t result = input[0];"
)
```

1. Capture 1000 traces with the same input.
2. Plot a histogram.
3. Explain the result.

Hints:
- A histogram with 10 "bins" at sample `x` can be created using: `np.histogram(data["trace"][:, x], bins=10, density=True)`
- The best plot for a histogram is a bar plot. Use `fig.add_trace(pgo.Bar(...))`.
- For CWLite XMEGA `x = 16` might be a good coordinate to look at.

</div>

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 2</div>

Generate traces using the following code:

```python
data = capture(
    ...
    code="volatile uint8_t result = input[0];"
)
```

1. Capture 300 traces: 100 with input `0x00`, 100 with input `0xF0`, and 100 with input `0xFF`.
2. Plot three histograms.
3. Explain the result.

</div>

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 3</div>

Create an extrapolation of the three histograms and plot as overlay together with the histograms.

Hints:
- `mu, std = scipy.stats.norm.fit(data)` extrapolates mean value and standard deviation of the given data
- `scipy.stats.norm.pdf(xs, mu, std)` calculates the probability density function for given mean, standard deviation, and `xs`.
- `xs = np.linspace(a, b)` generates an array between `a` and `b` with a default of `num=50` datapoints.

</div>

Looking at the exercises before we see that a "template" is nothing else then a mean and standard deviation value.
Usually multiple data points are needed to have a good template.
In a **multivariate Gaussian normal distribution** the terms are: **mean-vector** and **covariance-matrix**.

## 3. Develop an attack for AES SBox Lookup

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 4</div>

Create templates for AES SBox Lookup:

1. Record 5000 traces of AES SBox Lookup with random input and random key.
2. Extract only the sample points xx, yy, zz when using `cwlitexmega`.
3. Group the  by the Hamming weight of the output of the SBox Lookup of the first byte.
4. Calculate _mean vector_ and _covariance matrix_ for each group. 
   The mean vector is of shape `(3, )` for each group.
   The covariance matrix is of shape `(3, 3)` for each group.
5. Create a `scipy.stats.multivariate_normal(mean_vector, covariance_matrix)` instance for each group.

Apply the templates:

1. Capture one trace with an unknown key and random input.
2. Apply the template from above: Call `.pdf()` for each group.
   The maximum value corresponds to the most likely group.

Optional:
- Most often a single trace is not enough to get a reliable result. 
  Capture more traces with random input. 
  Apply the template on each trace using `logpdf` and sum the result.

</div>

## 4. Find Points of Interest

In the previous attack we assumed to know which positions are interesting.
Points of Interest (POIs) can be found e.g. with the following method:

1. Capture traces with random input and random key
2. Group the traces by the Hamming weight of the output of the SBox Lookup of the first byte
3. Calculate the sum of the pairwise squared difference, i.e. $\sum_{i,j=0}^8 (\bar t_i - \bar t_j)^2$ where $\bar t_i$ is the mean of all traces of the group with Hamming weight $i$.
4. Take the sample coordinates of 3 - 5 maximum points.

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 5</div>

Repeat the template attack but calculate the POIs.

</div>

## 5. Template attack using `sklearn.discriminant_analysis` and `lascar`

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 6</div>

Repeat the template attack using `sklearn.discriminant_analysis` and `lascar` following https://github.com/Ledger-Donjon/lascar/blob/master/examples/base/profiled-attack.py

</div>

In [ ]:
import sklearn
import sklearn.discriminant_analysis

In [ ]:
fig = pgo.Figure()
for d in data[:100]:
    fig.add_trace(pgo.Scatter(y=d["trace"]))
fig.show()


In [ ]:
traces_per_input = [[] for _ in range(9)]
for value in data:
    traces_per_input[lascar.hamming(lascar.tools.aes.sbox[value["input"][0] ^ value["input"][16]])].append(value["trace"])
    # traces_per_input[value["input"][0] ^ value["input"][16]].append(value["trace"])
means_per_input = [np.average(trace, 0) for trace in traces_per_input]

# Calculate sum of absolute differences
diffs = np.zeros(means_per_input[0].shape[0])
for mean_i in means_per_input:
    for mean_j in means_per_input:
        diffs += np.abs(mean_i - mean_j)
    
fig = pgo.Figure()
fig.add_trace(pgo.Scatter(y=diffs))
fig.show()

In [ ]:
# partition_function must take 1 argument: the value returned by the container at each trace
def partition_function(value):
    return lascar.hamming(lascar.tools.aes.sbox[
        value["input"][0] ^ value["input"][16]
    ])

classifier_qda = sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis(store_covariance=True)

classifier_profile_engine_qda = lascar.ProfileEngine(
    "profile qda",
    classifier=classifier_qda,
    partition_function=partition_function,
    partition_range=range(9),
)

trace = lascar.TraceBatchContainer(data["trace"], data)
trace.leakage_section = [89, 93, 97, 109, 113]
# trace.leakage_section = [45, 46, 47, 48, 49, 50]
session = lascar.Session(
    trace,
    engine=classifier_profile_engine_qda,
).run()


In [ ]:
data_attack = capture(
    platform="cwlitexmega",
    number_of_traces=50,
    number_of_samples=1000,
    fromfile=os.path.abspath("../lecture_3/sbox_lookup.c"),
    inputfunction=lambda _: [random.randint(0, 255)] + 15 * [0] + 16 * [0xab],
)

In [ ]:
# The sensitive value with guess hypothesis
def sensitive_value_with_guess(value, guess):
    return lascar.hamming(lascar.tools.aes.sbox[value["input"][0] ^ guess])
    # return guess

classifier_match_engine_qda = lascar.MatchEngine(
    "match qda",
    classifier=classifier_qda,
    selection_function=sensitive_value_with_guess,
    guess_range=range(256),
)

trace = lascar.TraceBatchContainer(data_attack["trace"][:10], data_attack[:10])
trace.leakage_section = [89, 93, 97, 109, 113]
# trace.leakage_section = [45, 46, 47, 48, 49, 50]
session = lascar.Session(
    trace,
    engine=classifier_match_engine_qda,
    # output_method=lascar.ScoreProgressionOutputMethod(classifier_match_engine_qda),
    output_method=lascar.ConsoleOutputMethod(classifier_match_engine_qda),
    output_steps=range(0, 50, 1),
).run(thread_on_update=False)

In [ ]:
classifier_qda.predict(data_attack["trace"][3, 46].reshape(-1, 1))

In [ ]:
clas = sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis(store_covariance=True)

In [ ]:
clas.fit([[0.5], [0.7], [0.6], [0.75]], [0, 1, 0, 1])
clas.predict([[0.7]])

In [ ]:
clas.fit(data["trace"][:, 12].reshape(-1, 1), list(map(lascar.hamming, data["input"][:, 0])))

In [ ]:
clas.predict(data["trace"][0, 12].reshape(-1, 1))

In [ ]:
import scipy
fig = pgo.Figure()
for mean, covar in zip(clas.means_, clas.covariance_):
    xs = np.linspace(mean[0] - 2 * covar[0, 0], mean[0] + 2 * covar[0, 0])
    s = pgo.Scatter(x=xs, y=scipy.stats.norm.pdf(xs, mean[0], covar[0, 0]))
    fig.add_trace(s)
fig.show()